## Version check

우리는 scikit learn 에서 제공하는 20NewsGroups data 를 이용하여 document classification 을 하는 multi-layer feed-foward neural network 를 만들어봅니다.

현재 실습의 torch 버전은 1.0.1 입니다.

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F

print(torch.__version__)

1.0.1.post2


## Load data

20NewsGroups 의 모든 데이터를 이용해도 되지만, 빠른 확인을 위하여 네 개의 카테고리만 이용합니다. 20NewsGroups 은 20 개의 카테고리로 분류된 뉴스 문서 집합입니다.

In [2]:
# Load training set and test set
categories = [
    'rec.sport.baseball',
    'soc.religion.christian',
    'comp.windows.x',
    'sci.space'
]

removals = ('headers', 'footers', 'quotes')

remove 에 'headers', 'footers', 'quotes' 를 넣으면 뉴스의 header 들이 제거 된 text 만 받을 수 있습니다.

In [3]:
newsgroups_train = fetch_20newsgroups(
    subset='train', remove=removals, categories=categories)

newsgroups_test = fetch_20newsgroups(
    subset='test', remove=removals, categories=categories)

여기서 text 와 category label 을 분리합니다.

In [4]:
data_train = newsgroups_train.data
y_train = newsgroups_train.target
data_test = newsgroups_test.data
y_test = newsgroups_test.target

## Vectorizing

TF-IDF vectorizer 를 이용하여 vectorizing 을 합니다. 이 때 학습 데이터 기준에서 word index 가 학습되도록 학습데이터에는 fit_transform 을 적용하고, 테스트 데이터에는 transform 을 적용합니다. 학습 데이터에 존재하지 않는 단어는 테스트 단어에서 벡터화 되지 않습니다.

In [5]:
vectorizer = TfidfVectorizer(min_df=5)
x_train = vectorizer.fit_transform(data_train)
x_test = vectorizer.transform(data_test)

print('train x = {}, y = {}'.format(x_train.shape, y_train.shape))
print('test  x = {}, y = {}'.format(x_test.shape, y_test.shape))

train x = (2382, 6535), y = (2382,)
test  x = (1584, 6535), y = (1584,)


## Model

Feed forward neural netowrk 를 만듭니다. torch.nn.Module 을 상속하여 init, forward 함수만 구현하면 됩니다.

init 을 구현할 때에는 Python 의 상속처럼 super().\_\_init\_\_() 을 실행해야 합니다. 우리는 3 개의 hidden layer 로 이뤄진 feed forward 를 만들 것입니다. 각각은 (input, hidden 1), (hidden 1, hidden 2), (hidden 2, classes) 의 크기로 이뤄진 weight matrix 를 지닙니다. bias 역시 학습하도록 설정합니다.

```python
self.fc_1 = nn.Linear(in_features = input_dim, out_features = hidden_1_dim, bias=True)
self.fc_2 = nn.Linear(in_features = hidden_1_dim, out_features = hidden_2_dim, bias=True)
self.fc_3 = nn.Linear(in_features = hidden_2_dim, out_features = n_classes, bias = True)
```

우리는 Linear layer 만을 만들었을 뿐, activation function 은 아직 만들지 않았습니다. hidden layer 1, 2 의 output 에 대하여 ReLU 를 적용합니다.

```python
def forward(self, x):
    out = F.relu(self.fc_1(x))
    out = F.relu(self.fc_2(out))
    return self.fc_3(out)
```

In [7]:
class FeedForwardNN(nn.Module):
    def __init__ (self, input_dim, hidden_1_dim, hidden_2_dim, n_classes):
        super(FeedForwardNN, self).__init__()
        self.fc_1 = nn.Linear(input_dim, hidden_1_dim, bias=True)
        self.fc_2 = nn.Linear(hidden_1_dim, hidden_2_dim, bias=True)
        self.fc_3 = nn.Linear(hidden_2_dim, n_classes, bias = True)

    def forward(self, x):
        out = F.relu(self.fc_1(x))
        out = F.relu(self.fc_2(out))
        return self.fc_3(out)

## Parameters

numpy.unique 는 numpy.ndarray 의 값의 set 입니다. 네 개의 카테고리가 각각 0, 1, 2, 3 으로 encoding 되어 있습니다.

In [8]:
np.unique(y_train)

array([0, 1, 2, 3])

학습에 이용할 패러매터를 정의합니다. epochs, mini batch size, hidden 1, hidden 2 의 크기를 정의합니다.

우리가 정의한 네트워크의 구조는 아래와 같습니다. 6,535 개의 단어로 표현된 문서가 128, 32 차원을 거쳐 4 개의 클래스로 분류됩니다.

    6535 - 128 - 32 - 4


In [9]:
# Parameters
epochs = 10
batch_size = 64

input_dim = x_train.shape[1]
hidden_1_dim = 128
hidden_2_dim = 32
n_classes = np.unique(y_train).shape[0]
n_data = x_train.shape[0]

print('layer = {} - {} - {} - {}'.format(
    input_dim, hidden_1_dim, hidden_2_dim, n_classes))
print('n data = {}'.format(n_data))
print('num mini-batch = {}'.format(n_data // batch_size))

layer = 6535 - 128 - 32 - 4
n data = 2382
num mini-batch = 37


## Model, loss function, optimizer

실제로 모델을 만듭니다. loss function (criterion) 과 optimizer 는 regression 과 비슷합니다. 단, classification 이기 때문에 Cross Entropy loss 를 이용합니다.

In [10]:
# Model
model = FeedForwardNN(
    input_dim,
    hidden_1_dim,
    hidden_2_dim,
    n_classes
)

# Parameter for the optimizer
learning_rate = 0.001

# Loss and optimizer
loss_func = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate
)

정의한 바와 같이 6535 - 128 - 32 구조의 hidden layer 를 지닌, 4 개의 클래스를 분류하는 neural network 가 만들어졌습니다.

In [11]:
print(model)

FeedForwardNN(
  (fc_1): Linear(in_features=6535, out_features=128, bias=True)
  (fc_2): Linear(in_features=128, out_features=32, bias=True)
  (fc_3): Linear(in_features=32, out_features=4, bias=True)
)


## Train function

epochs 만큼 iteration 을 돕니다. 각 epoch 마다의 누적 loss 를 저장하기 위한 준비를 합니다.

```python
for epoch in range(epochs):
    loss_sum = 0
    # TODO
```

mini batch 는 데이터의 일부로 loss 를 계산한 다음, 이를 반영하여 weight parameter 를 학습하는 것입니다. 데이터의 크기가 100 일 때, 한 번에 5 개의 데이터를 이용한다면 총 20 번의 mini batch 를 이용한 학습이 됩니다. 이는 다음처럼 구현합니다. begin (b), end (e) index 를 만들고, x 와 y 에서 b 부터 e 까지 slicing 을 합니다. 이 때 x 의 형식은 scipy.sparse 입니다. 

```python
for i in range(n_data // batch_size):

    b = i * batch_size
    e = min(n_data, (i+1) * batch_size)

    x_batch = x_train[b:e] # type : sparse matrix
    y_batch = y_train[b:e] # type : numpy.ndarray
```

이를 먼저 numpy.ndarray 로 변환한 뒤, torch.Tensor 로 변환합니다. 한 번에 x_train 을 scipy.sparse 에서 numpy.ndarray 로 변환하면 sparse matrix 가 dense matrix 로 변하면서 지나치게 많은 메모리를 소모하게 됩니다. 

x 는 TF-IDF 값이기 때문에 소수값을 포함합니다. 이는 torch.FloatTensor 로 구현합니다. label 은 정수이기 때문에 torch.LongTensor 로 감쌉니다.

```python
for i in range(n_data // batch_size):
    # ...
    x_batch = torch.FloatTensor(x_batch.todense())
    y_batch = torch.LongTensor(y_batch)
```

항상 optimizer 에 이전 step 에서 이용한 gradient 를 지운 뒤, forwarding, back-propagation 을 합니다.

```python
for i in range(n_data // batch_size):
    # ...
    optimizer.zero_grad()
```

prediction, loss 계산, 그리고 back propagation 을 수행합니다.

```python
for i in range(n_data // batch_size):
    # ...
    x_pred = model(x_batch)
    loss = criterion(x_pred, y_batch)
    loss.backward()
    optimizer.step()
```

loss 의 data 에는 loss 값이 저장되어 있습니다. 이는 torch.Tensor 이니 numpy() 를 이용하여 숫자로 변환하여 loss_sum 에 누적합니다.

```python
for i in range(n_data // batch_size):
    # ...
    loss_sum += loss.data.numpy()
```

매 epoch 마다 loss 가 줄어듦을 확인할 수 있습니다.

In [12]:
def train(x_train, y_train, model, loss_func, optimizer, batch_size):

    n_data = x_train.shape[0]

    # Loop over all epochs
    for epoch in range(epochs):

        loss_sum = 0

        for i in range(n_data // batch_size):

            # select mini-batch data
            b = i * batch_size
            e = min(n_data, (i+1) * batch_size)
            x_batch = x_train[b:e] # type : sparse matrix
            y_batch = y_train[b:e] # type : numpy.ndarray

            # scipy.sparse -> numpy.ndarray -> torch.Tensor
            x_batch = torch.FloatTensor(x_batch.todense())
            y_batch = torch.LongTensor(y_batch)

            # Forward -> Backward -> Optimize
            optimizer.zero_grad() # Make the gradient buffer zero
            x_pred = model(x_batch)
            loss = loss_func(x_pred, y_batch)
            loss.backward()
            optimizer.step()

            # cumulate loss
            loss_sum += loss.data.numpy()

            if i % 10 == 0:
                loss_tmp = loss_sum / (i+1)
                print('\repoch = {}, batch = {}, training loss = {}'.format(
                    epoch, i+1, '%.3f' % loss_tmp), end='')

        print('\r## epoch = {}, training loss = {}'.format(epoch, '%.3f' % (loss_sum / (i+1)) ))

    return model

model = train(x_train, y_train, model, loss_func, optimizer, batch_size)

## epoch = 0, training loss = 1.340s = 1.357
## epoch = 1, training loss = 0.878s = 0.934
## epoch = 2, training loss = 0.295s = 0.310
## epoch = 3, training loss = 0.121s = 0.121
## epoch = 4, training loss = 0.072s = 0.069
## epoch = 5, training loss = 0.053s = 0.050
## epoch = 6, training loss = 0.043s = 0.040
## epoch = 7, training loss = 0.038s = 0.035
## epoch = 8, training loss = 0.035s = 0.032
## epoch = 9, training loss = 0.033s = 0.030


## Training accuracy test

테스트 데이터에 대하여 정확도를 측정합니다. 데이터의 크기가 얼마 크지 않기 때문에 한 번에 dense matrix 로 변환하였습니다. 그리고 torch.Tensor 로 변환하여 모델에 입력합니다.

```python
x_test_tensor = torch.FloatTensor(x_test.todense())
```

forward 함수가 call 함수로 지정되어 있기 때문에 아래 두 구문은 같은 기능을 수행합니다.

```python
y_test_pred = model(x_test_tensor)
y_test_pred = model.forward(x_test_tensor)
```

그 결과 1584 개의 데이터에 대하여 각각 4 개의 클래스와의 softmax 값이 계산됩니다.

In [13]:
x_test.shape

(1584, 6535)

In [14]:
# Make test batch data
x_test_tensor = torch.FloatTensor(x_test.todense())
y_test_pred = model(x_test_tensor)

print(y_test_pred.size())

torch.Size([1584, 4])


torch.max 는 torch.Tensor 에 대하여 dim 기준으로 최대값과 그 값을 지니는 index 를 출력합니다. dim=1 이기 때문에 [1584, 4] size 에서 4 인, column 기준으로 max 값을 찾습니다. return 은 max value 와 index 로 나뉘어져 됩니다.

In [15]:
score, predicted = torch.max(y_test_pred.data, dim=1)

print(score.size())
print(predicted.size())

torch.Size([1584])
torch.Size([1584])


In [16]:
score[:3]

tensor([1.7198, 9.5568, 8.6373])

In [17]:
predicted[:3]

tensor([3, 3, 2])

In [18]:
print(y_test.shape)
print(predicted.numpy().shape)

(1584,)
(1584,)


numpy.ndarray 를 이용하여 predicted 된 값과 label 이 같은 indices 를 찾습니다. 그 길이만큼 제대로된 prediction 을 수행한 것입니다.

In [19]:
n_correct = np.where(y_test == predicted.numpy())[0].shape[0]
accuracy = n_correct / y_test.shape[0]

print('accuracy = {}'.format(accuracy))

accuracy = 0.9059343434343434
